In [1]:
# Import dependencies and setup
import pandas as pd
import requests
import json
from json import loads, dumps
from pprint import pprint

# Get TEAMS data

In [2]:
# Make an HTTP GET request to the ESPN API

# Create a list of team numbers
team_numbers = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,33,34]
team_number = team_numbers[0]

# Create empty lists to store team and team records data
teams = []
teams_records = []

# Create the URL details
base_url = "http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2022/teams/"
tail_url = "?lang=en&region=us"
final_url = base_url + str(team_number) + tail_url

# Get the data from the final URL
response = requests.get(final_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    
    # Loop through the teams and store relevant information
    for x in range(0,32):

        # Create the URL for the team and get the JSON data
        team_number = team_numbers[x]
        final_url = base_url + str(team_number) + tail_url
        response = requests.get(final_url)
        data = response.json()
        
        # Retrieve the relevant information
        team_id = int(data['id'])
        team_name = data['shortDisplayName']
        team_nickname = data['nickname']
        team_location = data['location']
        team_abbreviation = data['abbreviation']
        team_displayName = data['displayName']
        team_isActive = data['isActive']
        team_isAllStar = data['isAllStar']
        team_logos = data['logos'][0]['href']
        
        # Save the data to the list
        teams.append({
            'id': team_id, 
            'shortDisplayName': team_name, 
            'nickname': team_nickname, 
            'location': team_location,
            'abbreviation': team_abbreviation, 
            'displayName': team_displayName, 
            'isActive': team_isActive, 
            'isAllStar': team_isAllStar, 
            'logos': team_logos})     

else:
    print("Error: Could not fetch data from the ESPN API")

In [3]:
# Loop through teams records and store relevant information

for x in range(0,32):

    # Create the URL details
    record_url = "http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2022/types/2/teams/" + str(team_numbers[x]) + "/record?lang=en&region=us"
    
    # Get the data from the record URL
    record_response = requests.get(record_url)
    record_data = record_response.json()
    
    # Get team id to link afterwards
    teamID = team_numbers[x]

    # Retrieve the relevant information
    OTLosses = record_data['items'][0]['stats'][0]['value']
    OTWins = record_data['items'][0]['stats'][1]['value']
    avgPointsAgainst = record_data['items'][0]['stats'][2]['value']
    avgPointsFor = record_data['items'][0]['stats'][3]['value']
    divisionWinPercent = record_data['items'][0]['stats'][6]['value']
    gamesPlayed = record_data['items'][0]['stats'][8]['value']
    losses = record_data['items'][0]['stats'][10]['value']
    ties = record_data['items'][0]['stats'][16]['value']
    wins = record_data['items'][0]['stats'][18]['value']
    winPercent = record_data['items'][0]['stats'][17]['value']
    points = record_data['items'][0]['stats'][12]['value']

    # Save the data to the list
    teams_records.append({
                        'id':teamID, 
                        'OTLosses': OTLosses, 
                        'OTWins': OTWins, 
                        'avgPointsAgainst': avgPointsAgainst, 
                        'avgPointsFor': avgPointsFor,
                         'divisionWinPercent': divisionWinPercent, 
                         'gamesPlayed': gamesPlayed, 
                         'losses': losses, 
                         'ties': ties, 
                         'wins': wins, 
                         'winPercent': winPercent, 
                         'points': points})


### Convert TEAMS data to DataFrame

In [4]:
# Create teams dataframe
teams_df = pd.DataFrame(teams)

# Show the dataframe
teams_df.head()

,id,shortDisplayName,nickname,location,abbreviation,displayName,isActive,isAllStar,logos
0,1,Falcons,Falcons,Atlanta,ATL,Atlanta Falcons,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/atl.png
1,2,Bills,Bills,Buffalo,BUF,Buffalo Bills,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/buf.png
2,3,Bears,Bears,Chicago,CHI,Chicago Bears,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/chi.png
3,4,Bengals,Bengals,Cincinnati,CIN,Cincinnati Bengals,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/cin.png
4,5,Browns,Browns,Cleveland,CLE,Cleveland Browns,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/cle.png


In [5]:
# Create teams records dataframe
teams_records_df = pd.DataFrame(teams_records)

# Show the dataframe
teams_records_df.head()

,id,OTLosses,OTWins,avgPointsAgainst,avgPointsFor,divisionWinPercent,gamesPlayed,losses,ties,wins,winPercent,points
0,1,0.0,1.0,22.705883,21.470589,0.333333,17.0,10.0,2.0,0.411765,0.0,-21.0
1,2,1.0,0.0,17.875000,28.437500,0.666667,16.0,3.0,7.0,0.812500,0.0,169.0
2,3,0.0,0.0,27.235294,19.176470,0.000000,17.0,14.0,-10.0,0.176471,0.0,-137.0
3,4,1.0,0.0,20.125000,26.125000,0.500000,16.0,4.0,8.0,0.750000,0.0,96.0
4,5,0.0,1.0,22.411764,21.235294,0.500000,17.0,10.0,-1.0,0.411765,0.0,-20.0


In [6]:
# Merge the two teams dataframes
teams_data_df = teams_df.merge(teams_records_df, on='id', how='left')

# Show the merged dataframe
teams_data_df.head()

,id,shortDisplayName,nickname,location,abbreviation,displayName,isActive,isAllStar,logos,OTLosses,OTWins,avgPointsAgainst,avgPointsFor,divisionWinPercent,gamesPlayed,losses,ties,wins,winPercent,points
0,1,Falcons,Falcons,Atlanta,ATL,Atlanta Falcons,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/atl.png,0.0,1.0,22.705883,21.470589,0.333333,17.0,10.0,2.0,0.411765,0.0,-21.0
1,2,Bills,Bills,Buffalo,BUF,Buffalo Bills,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/buf.png,1.0,0.0,17.875000,28.437500,0.666667,16.0,3.0,7.0,0.812500,0.0,169.0
2,3,Bears,Bears,Chicago,CHI,Chicago Bears,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/chi.png,0.0,0.0,27.235294,19.176470,0.000000,17.0,14.0,-10.0,0.176471,0.0,-137.0
3,4,Bengals,Bengals,Cincinnati,CIN,Cincinnati Bengals,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/cin.png,1.0,0.0,20.125000,26.125000,0.500000,16.0,4.0,8.0,0.750000,0.0,96.0
4,5,Browns,Browns,Cleveland,CLE,Cleveland Browns,True,False,https://a.espncdn.com/i/teamlogos/nfl/500/cle.png,0.0,1.0,22.411764,21.235294,0.500000,17.0,10.0,-1.0,0.411765,0.0,-20.0


### Convert TEAMS data to JSON and CSV

In [8]:
# Convert the teams data to JSON
teams_data_df.to_json('../data/Outputs_JSON//teams.json', indent=4, orient='records')

In [10]:
# Convert the teams data to CSV
teams_data_df.to_csv('../data/Outputs_CSV//teams.csv')

# Get ATHLETES data

In [43]:
# Create empty list to store athletes data for offense, defense and special players
athletes = []

# Loop through the roster to get athletes information
for x in range(0,32):
    roster_url = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/" + str(team_numbers[x]) + "/roster"
    roster_response = requests.get(roster_url)
    roster_data = roster_response.json()

    # Retrieve the relevant team information
    team_id_athletes =  roster_data['team']['id']
    team_name_athletes =  roster_data['team']['displayName']

    # Retrieve the relevant coach information
    coach = roster_data['coach']
    coach_fname = roster_data['coach'][0]['firstName']
    coach_lname = roster_data['coach'][0]['lastName']
    coach_xp = roster_data['coach'][0]['experience']

    # Get the length to determine the total number of athletes in each category
    offense = len(roster_data['athletes'][0]['items'])
    defense = len(roster_data['athletes'][1]['items'])
    special = len(roster_data['athletes'][2]['items'])
    
    # Create an empty list to store id information
    ids = []


    # Get data for Offense Athletes
    # Loop through the number of offense athletes
    for y in range(offense):
        try:
            # Retrieve id information and append it to the list
            offense_id = roster_data['athletes'][0]['items'][y]['id']
            teamID = team_numbers[x]
            teamName = team_name_athletes
            ids.append(offense_id)

            # Retrieve the relevant information
            type = roster_data['athletes'][0]['position']
            offense_name = roster_data['athletes'][0]['items'][y]['fullName']
            offense_weight = roster_data['athletes'][0]['items'][y]['weight']
            offense_height = roster_data['athletes'][0]['items'][y]['height']
            offense_age = roster_data['athletes'][0]['items'][y]['age']
            offense_birthCity = roster_data['athletes'][0]['items'][y]['birthPlace']['city']
            offense_birthState = roster_data['athletes'][0]['items'][y]['birthPlace']['state']
            offense_birthCountry = roster_data['athletes'][0]['items'][y]['birthPlace']['country']
            offense_weight = roster_data['athletes'][0]['items'][y]['weight']
            offense_position = roster_data['athletes'][0]['items'][y]['position']['name']
            offense_jersey = roster_data['athletes'][0]['items'][y]['jersey']
            offense_xp = roster_data['athletes'][0]['items'][y]['experience']['years']
            offense_status = roster_data['athletes'][0]['items'][y]['status']['type']
            offense_headshot = roster_data['athletes'][0]['items'][y]['headshot']['href']

            # Save the data to the list
            athletes.append({
                'type': type,
                'teamID': teamID,
                'teamName': teamName,
                'name': offense_name, 
                'weight': offense_weight, 
                'height': offense_height, 
                'age': offense_age, 
                'birthCity': offense_birthCity, 
                'birthState': offense_birthState, 
                'birthCountry': offense_birthCountry, 
                'weight': offense_weight,
                'position': offense_position, 
                'jersey': offense_jersey, 
                'xp': offense_xp, 
                'status': offense_status, 
                'headshot': offense_headshot
                })

        # Handle any Key Errors
        except KeyError:
            print('N/A')
            athletes.append({
                'type': type,
                'teamID': teamID,
                'teamName': teamName,
                'name': offense_name, 
                'weight': offense_weight, 
                'height': offense_height, 
                'age': offense_age, 
                'birthCity': offense_birthCity, 
                'birthState': offense_birthState, 
                'birthCountry': offense_birthCountry, 
                'weight': offense_weight,
                'position': offense_position, 
                'jersey': offense_jersey, 
                'xp': offense_xp, 
                'status': offense_status, 
                'headshot': offense_headshot
                })
            continue


    # Get data for Defense Athletes
    # Loop through the number of defense athletes
    for y in range(defense):
        try:
            # Retrieve id information and append it to the list
            defense_id = roster_data['athletes'][1]['items'][y]['id']
            ids.append(defense_id)

            # Retrieve the relevant information
            type = roster_data['athletes'][1]['position']
            defense_name = roster_data['athletes'][1]['items'][y]['fullName']
            defense_weight = roster_data['athletes'][1]['items'][y]['weight']
            defense_height = roster_data['athletes'][1]['items'][y]['height']
            defense_age = roster_data['athletes'][1]['items'][y]['age']
            defense_birthCity = roster_data['athletes'][1]['items'][y]['birthPlace']['city']
            defense_birthState = roster_data['athletes'][1]['items'][y]['birthPlace']['state']
            defense_birthCountry = roster_data['athletes'][1]['items'][y]['birthPlace']['country']
            defense_weight = roster_data['athletes'][1]['items'][y]['weight']
            defense_position = roster_data['athletes'][1]['items'][y]['position']['name']
            defense_jersey = roster_data['athletes'][1]['items'][y]['jersey']
            defense_xp = roster_data['athletes'][1]['items'][y]['experience']['years']
            defense_status = roster_data['athletes'][1]['items'][y]['status']['type']
            defense_headshot = roster_data['athletes'][1]['items'][y]['headshot']['href']

            # Save the data to the list
            athletes.append({
                'type': type,
                'teamID': teamID,
                'teamName': teamName,
                'name': defense_name, 
                'weight': defense_weight, 
                'height': defense_height, 
                'age': defense_age, 
                'birthCity': defense_birthCity, 
                'birthState': defense_birthState, 
                'birthCountry': defense_birthCountry, 
                'weight': defense_weight,
                'position': defense_position, 
                'jersey': defense_jersey, 
                'xp': defense_xp, 
                'status': defense_status, 
                'headshot': defense_headshot})

        # Handle any Key Errors  
        except KeyError:
            print('N/A')
            athletes.append({
                'type': type,
                'teamID': teamID,
                'teamName': teamName,
                'name': defense_name, 
                'weight': defense_weight, 
                'height': defense_height, 
                'age': defense_age, 
                'birthCity': defense_birthCity, 
                'birthState': defense_birthState, 
                'birthCountry': defense_birthCountry, 
                'weight': defense_weight,
                'position': defense_position, 
                'jersey': defense_jersey, 
                'xp': defense_xp, 
                'status': defense_status, 
                'headshot': defense_headshot})
            continue

    for y in range(special):
        try:
            # Retrieve id information and append it to the list
            special_id = roster_data['athletes'][2]['items'][y]['id']
            ids.append(special_id)

            # Retrieve the relevant information
            type = roster_data['athletes'][2]['position']
            special_name = roster_data['athletes'][2]['items'][y]['fullName']
            special_weight = roster_data['athletes'][2]['items'][y]['weight']
            special_height = roster_data['athletes'][2]['items'][y]['height']
            special_age = roster_data['athletes'][2]['items'][y]['age']
            special_birthCity = roster_data['athletes'][2]['items'][y]['birthPlace']['city']
            special_birthState = roster_data['athletes'][2]['items'][y]['birthPlace']['state']
            special_birthCountry = roster_data['athletes'][2]['items'][y]['birthPlace']['country']
            special_weight = roster_data['athletes'][2]['items'][y]['weight']
            special_position = roster_data['athletes'][2]['items'][y]['position']['name']
            special_jersey = roster_data['athletes'][2]['items'][y]['jersey']
            special_xp = roster_data['athletes'][2]['items'][y]['experience']['years']
            special_status = roster_data['athletes'][2]['items'][y]['status']['type']
            special_headshot = roster_data['athletes'][2]['items'][y]['headshot']['href']

            # Save the data to the list
            athletes.append({
                'type': type,
                'teamID': teamID,
                'teamName': teamName,
                'name': special_name, 
                'weight': special_weight, 
                'height': special_height, 
                'age': special_age, 
                'birthCity': special_birthCity, 
                'birthState': special_birthState, 
                'birthCountry': special_birthCountry, 
                'weight': special_weight,
                'position': special_position, 
                'jersey': special_jersey, 
                'xp': special_xp, 
                'status': special_status, 
                'headshot': special_headshot})

        # Handle any Key Errors
        except KeyError:
            print('N/A')
            athletes.append({
                'type': type,
                'teamID': teamID,
                'teamName': teamName,
                'name': special_name, 
                'weight': special_weight, 
                'height': special_height, 
                'age': special_age, 
                'birthCity': special_birthCity, 
                'birthState': special_birthState, 
                'birthCountry': special_birthCountry, 
                'weight': special_weight,
                'position': special_position, 
                'jersey': special_jersey, 
                'xp': special_xp, 
                'status': special_status, 
                'headshot': special_headshot})
            continue

# print(athletes)

N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A


In [44]:
print(athletes[0])

{'type': 'offense', 'teamID': 1, 'teamName': 'Atlanta Falcons', 'name': 'Josh Ali', 'weight': 193.0, 'height': 72.0, 'age': 24, 'birthCity': 'Hollywood', 'birthState': 'FL', 'birthCountry': 'USA', 'position': 'Wide Receiver', 'jersey': '80', 'xp': 0, 'status': 'active', 'headshot': 'https://a.espncdn.com/i/headshots/nfl/players/full/3051890.png'}


### Convert ATHLETES data to DataFrame

In [45]:
# Convert the athletes data into dataframe
athletes_df = pd.DataFrame(athletes)

In [49]:
# Show the athletes dataframe
athletes_df.head()

,type,teamID,teamName,name,weight,height,age,birthCity,birthState,birthCountry,position,jersey,xp,status,headshot
0,offense,1,Atlanta Falcons,Josh Ali,193.0,72.0,24,Hollywood,FL,USA,Wide Receiver,80,0,active,https://a.espncdn.com/i/headshots/nfl/players/...
1,offense,1,Atlanta Falcons,Tyler Allgeier,220.0,71.0,23,Fontana,CA,USA,Running Back,25,0,active,https://a.espncdn.com/i/headshots/nfl/players/...
2,offense,1,Atlanta Falcons,B.J. Baylor,202.0,70.0,24,Wharton,TX,USA,Running Back,36,0,active,https://a.espncdn.com/i/headshots/nfl/players/...
3,offense,1,Atlanta Falcons,Jared Bernhardt,195.0,73.0,25,Longwood,FL,USA,Wide Receiver,83,0,active,https://a.espncdn.com/i/headshots/nfl/players/...
4,offense,1,Atlanta Falcons,Drew Dalman,300.0,75.0,24,Salinas,CA,USA,Center,67,2,active,https://a.espncdn.com/i/headshots/nfl/players/...


### Convert ATHLETES data to JSON and CSV

In [50]:
# Convert the athletes data to JSON
athletes_df.to_json('../data/Outputs_JSON//athletes.json', indent=4, orient='records')

In [52]:
# Convert the athletes data to CSV
athletes_df.to_csv('../data/Outputs_CSV/athletes.csv')

# Get the STATISTICS data

In [53]:
# Get team the statistics data

# Read the Excel file containing the data
stats = pd.read_csv('../data/Resources/nfl_team_stats_2002-2022.csv')

### Convert STATISTICS data to DataFrame

In [54]:
# Convert the data to a dataframe
stats_df = pd.DataFrame(stats)

# Show the dataframe
stats_df.head()

,date,away,home,first_downs_away,first_downs_home,third_downs_away,third_downs_home,fourth_downs_away,fourth_downs_home,passing_yards_away,...,redzone_away,redzone_home,drives_away,drives_home,def_st_td_away,def_st_td_home,possession_away,possession_home,score_away,score_home
0,2002-09-05,49ers,Giants,13,21,4-12,9-16,0-0,0-1,166,...,0-8,0-6,13,15,0,0,27:32,32:28,16,13
1,2002-09-08,Jets,Bills,18,26,2-8,7-17,0-0,2-2,193,...,0-9,0-8,17,18,2,0,21:06,39:08,37,31
2,2002-09-08,Vikings,Bears,19,20,5-13,7-13,0-0,0-0,228,...,0-7,0-6,17,17,0,0,31:31,28:29,23,27
3,2002-09-08,Chargers,Bengals,27,13,6-10,4-11,0-0,0-0,160,...,0-7,0-5,11,15,0,0,37:48,22:12,34,6
4,2002-09-08,Chiefs,Browns,24,24,5-11,4-11,0-0,0-0,276,...,0-10,0-10,20,17,0,0,30:16,29:44,40,39


### Convert STATISTICS data to JSON and CSV

In [ ]:
# Convert the statistics data to JSON
stats_df.to_json('../data/Outputs_JSON/stats.json', indent=4, orient='records')

In [ ]:
# Convert the statistics data to CSV
stats_df.to_csv('../data/Outputs_CSV/stats.csv')